In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn import svm
import pandas as pd
from IPython.display import display

from __future__ import print_function
import os
import random
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.utils as vutils
import math

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r gdrive/MyDrive/QuSense/Papers/Simulated\ Data\ from\ Sarovar\ et\ al. sim_data

cp: cannot open 'gdrive/MyDrive/QuSense/Papers/Simulated Data from Sarovar et al./Format of Data.gdoc' for reading: Operation not supported
cp: cannot open 'gdrive/MyDrive/QuSense/Papers/Simulated Data from Sarovar et al./Data Description.gdoc' for reading: Operation not supported


In [ ]:
d = np.load("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates/CTexts_0.npy", allow_pickle=True)

In [54]:
def setup_data(dir, num_circuits = 20, file_prefix = "FCTexts_", dir_with_gates = True, separate_bits = "False", norm = 'max_norm'):
  onlyfiles = next(os.walk(dir))[2] #dir is your directory path as string

  # Currently, we only count files with frequencies
  if dir_with_gates:
    num_files = int(math.ceil((len(onlyfiles) - 1)/3))
  else:
    num_files = len(onlyfiles)

  for i in range(num_files): # Only FC files matter and there is an extra IDs file
    # Give unique indexes to each circuit in each file
    idx = i*num_circuits
    d = np.load(dir + "/" + file_prefix + str(i) + ".npy", allow_pickle=True)
    circuits = np.concatenate(d[:,0]).reshape(d.shape[0],2) + np.array([idx,idx+ int(num_circuits/2)]) 

    # Separate out cicruits and freq
    d = d[:,1:]

    # Stack
    if i == 0:
      det_c = np.copy(d)
      circ = np.copy(circuits)
    else:
      det_c = np.vstack((det_c, d))
      circ = np.vstack((circ, circuits))

  # Separate out frequencies by setting on each bit (instead of settings on pairs)
  if separate_bits == "True":
    det_c_copy = np.empty(det_c.shape)

    # 0 on qubit_0
    det_c_copy[:,0] = det_c[:,0] + det_c[:,2]

    # 1 on qubit_0
    det_c_copy[:,1] = det_c[:,1] + det_c[:,3]

    # 0 on qubit_1
    det_c_copy[:,2] = det_c[:,0] + det_c[:,1]

    # 1 on qubit_1
    det_c_copy[:,3] = det_c[:,2] + det_c[:,3]

    det_c = det_c_copy



  # Normalize frequencies
  if norm == "max_norm":
    det_c = det_c/np.max(det_c)
  elif norm == "col_max_norm":
    det_c = det_c/det_c.max(axis = 0)
  elif norm == "row_norm":
    det_c = det_c/np.sum(det_c[0,:])

  # Used only for testing
  elif norm == "no_norm":
    det_c = det_c

  return circ, det_c

def create_labels(label, size):
  return np.full((size,), label)

def create_dataset(features, label):
  y = create_labels(label, features.shape[0])
  return features, y

def combine_datasets(x1, y1, x2, y2):
  return np.vstack((x1,x2)), np.concatenate((y1,y2))

def binary_comparison_data_pipeline(freq_1, freq_2, test_frac = 0.2):
  x1, y1 = create_dataset(freq_1, 1)
  x2, y2 = create_dataset(freq_2, 0)
  x, y = combine_datasets(x1,y1, x2, y2)
  X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

def svm_expt(freq_1, freq_2, test_size = 0.2):
  X_train, X_test, y_train, y_test = binary_comparison_data_pipeline(freq_1, freq_2, test_frac = test_size)

  clf = svm.SVC()
  clf.fit(X_train, y_train)

  test_acc = (clf.predict(X_test) == y_test).mean()

  return test_acc

# Apply binary expt to all possible combinations of data_list and pretty print
def create_expt_matrix(dataset_list, data_list, binary_expt):
  num_datasets = len(data_list)
  expt_matrix = np.zeros((num_datasets, num_datasets))
  for i in range(num_datasets):
    for j in range(num_datasets):
      if (j >= i) :
        expt_matrix[i][j] = binary_expt(data_list[i][1], data_list[j][1])

  # pretty printing
  df = pd.DataFrame(expt_matrix, index=dataset_list, columns=dataset_list)
  display(df)

  return expt_matrix

In [49]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", norm = "row_norm")

In [50]:
ideal_freq.sum(axis = 1)

array([1.0, 1.0, 1.0, ..., 1.0, 1.0, 1.0], dtype=object)

In [46]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", separate_bits = "True", norm = "row_norm")

In [48]:
ideal_freq.sum(axis = 1)

array([1., 1., 1., ..., 1., 1., 1.])

In [53]:
xt2WithGates_circuits, xt2WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates")

100


KeyboardInterrupt: ignored

## Normalization Schemes


1.   max_norm - takes the maximum over the entire frequency matrix (across all circuit settings and all result bit settings)
2.   col_max_norm - takes the columnwise maximum (i.e max across all circuit settings for a particular result setting)
3. row_norm - uses the row-wise sum (i.e total number of times a circuit setting was simulated)



## Expts with bit frequencies not separated

In [ ]:
ideal_circuits, ideal_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates")
a_circuits, a_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 1 (a)", dir_with_gates = False)
b_circuits, b_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 2 (b)", dir_with_gates = False)
c_circuits, c_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Detection Crosstalk (c)",num_circuits=40, dir_with_gates = False)
detx400_circuits, detx400_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX400", num_circuits=40)
detxWithGates_circuits, detxWithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates", num_circuits=40)
detx100_circuits, detx100_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX100")
xt1WithGates_circuits, xt1WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates")
xt2WithGates_circuits, xt2WithGates_freq = setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT2WithGates")

In [56]:
datasets = ["IdealGates", "Operation Crosstalk 1 (a)", "Operation Crosstalk 2 (b)", "Detection Crosstalk (c)", "XT1WithGates", "XT2WithGates", "DetX400", "DetXWithGates", "DetX100"]

### max_norm

In [57]:
data_list = []
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates"))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 1 (a)", dir_with_gates = False))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 2 (b)", dir_with_gates = False))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Detection Crosstalk (c)",num_circuits=40, dir_with_gates = False))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates"))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT2WithGates"))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX400", num_circuits=40))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates", num_circuits=40))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX100"))

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

### col_max_norm

In [ ]:
data_list = []
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 1 (a)", dir_with_gates = False, norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 2 (b)", dir_with_gates = False, norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Detection Crosstalk (c)",num_circuits=40, dir_with_gates = False, norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates", norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT2WithGates", norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX400", num_circuits=40, norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates", num_circuits=40, norm = 'col_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX100", norm = 'col_max_norm'))

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

### row_norm

In [ ]:
data_list = []
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 1 (a)", dir_with_gates = False, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 2 (b)", dir_with_gates = False, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Detection Crosstalk (c)",num_circuits=40, dir_with_gates = False, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates", norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT2WithGates", norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX400", num_circuits=40, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates", num_circuits=40, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX100", norm = 'row_max_norm'))

In [ ]:
svm_expt_matrix = create_expt_matrix(datasets, data_list, svm_expt)

## Expts with bit frequencies separated

### max_norm

In [ ]:
data_list = []
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates", norm = 'max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 1 (a)", dir_with_gates = False, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Operation Crosstalk 2 (b)", dir_with_gates = False, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/Detection Crosstalk (c)",num_circuits=40, dir_with_gates = False, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT1WithGates", norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/XT2WithGates", norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX400", num_circuits=40, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates", num_circuits=40, norm = 'row_max_norm'))
data_list.append(setup_data("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetX100", norm = 'row_max_norm'))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
(clf.predict(X_test) == y_test).mean()

0.99325

### col_max_norm

array([[[0, 4],
        [0, 5],
        [0, 2],
        [0, 6],
        [0, 3]],

       [[1, 3],
        [1, 5],
        [1, 2],
        [1, 3],
        [1, 1]],

       [[2, 0],
        [2, 9],
        [2, 2],
        [2, 3],
        [2, 9]],

       [[3, 6],
        [3, 1],
        [3, 9],
        [3, 2],
        [3, 1]],

       [[4, 8],
        [4, 7],
        [4, 8],
        [4, 8],
        [4, 8]],

       [[5, 3],
        [5, 2],
        [5, 8],
        [5, 5],
        [5, 6]],

       [[6, 6],
        [6, 1],
        [6, 4],
        [6, 9],
        [6, 6]],

       [[7, 6],
        [7, 5],
        [7, 3],
        [7, 5],
        [7, 9]],

       [[8, 0],
        [8, 5],
        [8, 6],
        [8, 6],
        [8, 3]],

       [[9, 0],
        [9, 0],
        [9, 2],
        [9, 6],
        [9, 7]]])

In [ ]:
d1 = np.load("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/IdealGates/Gates_0.npy", allow_pickle=True)

In [ ]:
#(2,10,30)
d1[0]

array([[2, 0, 2, 1, 0, 0, 1, 0, 0, 1, 1, 2, 1, 1, 2, 0, 2, 0, 1, 2, 1, 1,
        1, 0, 2, 2, 2, 1, 1, 0],
       [1, 0, 2, 0, 1, 2, 0, 0, 1, 1, 2, 0, 0, 2, 0, 2, 2, 0, 1, 1, 1, 2,
        1, 1, 1, 1, 1, 2, 1, 2],
       [0, 2, 2, 0, 2, 1, 0, 1, 2, 0, 2, 2, 2, 2, 0, 1, 1, 0, 1, 1, 1, 1,
        0, 2, 2, 0, 0, 2, 0, 0],
       [2, 0, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 2, 1, 1, 2, 0, 1,
        0, 0, 1, 2, 2, 2, 0, 2],
       [2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 1, 1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1,
        2, 2, 0, 0, 0, 0, 2, 2],
       [2, 0, 1, 2, 2, 2, 0, 0, 2, 0, 1, 0, 0, 0, 1, 2, 0, 2, 1, 2, 0, 1,
        0, 1, 1, 0, 2, 0, 1, 1],
       [2, 1, 0, 2, 2, 1, 0, 0, 2, 0, 0, 2, 0, 1, 2, 2, 2, 0, 0, 1, 2, 1,
        2, 2, 2, 2, 0, 2, 1, 2],
       [2, 0, 0, 2, 1, 1, 1, 0, 0, 1, 2, 1, 2, 2, 2, 1, 0, 1, 0, 2, 1, 0,
        2, 1, 0, 2, 0, 0, 0, 1],
       [2, 0, 0, 2, 2, 0, 0, 2, 2, 1, 2, 1, 0, 2, 1, 1, 0, 2, 2, 0, 0, 2,
        2, 2, 0, 2, 0, 2, 2, 0],
       [2, 0, 2, 0, 0, 1, 1, 1, 0, 0,

### row_norm

In [ ]:
d2 = np.load("gdrive/MyDrive/QuSense/Papers/Simulated-Data-Sarovar/DetXWithGates/FCTexts_0.npy", allow_pickle=True)

In [ ]:
d2[:20]

array([[array([ 0, 16]), 24868, 25069, 25371.0, 24692.0],
       [array([ 0, 13]), 24851, 25133, 24861.0, 25155.0],
       [array([ 0, 13]), 25051, 25099, 24966.0, 24884.0],
       [array([0, 1]), 47414, 47773, 2410.0, 2403.0],
       [array([ 0, 11]), 25067, 25063, 25007.0, 24863.0],
       [array([0, 9]), 2365, 2428, 47872.0, 47335.0],
       [array([0, 4]), 25077, 24917, 25162.0, 24844.0],
       [array([0, 7]), 25061, 24958, 25172.0, 24809.0],
       [array([0, 9]), 2302, 2400, 47670.0, 47628.0],
       [array([0, 0]), 24951, 24746, 25288.0, 25015.0],
       [array([ 1, 17]), 25068, 24999, 24881.0, 25052.0],
       [array([ 1, 17]), 24944, 25153, 24771.0, 25132.0],
       [array([1, 0]), 25117, 25080, 24615.0, 25188.0],
       [array([ 1, 12]), 2419, 2391, 47644.0, 47546.0],
       [array([ 1, 15]), 47657, 47567, 2389.0, 2387.0],
       [array([1, 7]), 25025, 24809, 24904.0, 25262.0],
       [array([ 1, 12]), 2333, 2315, 47814.0, 47538.0],
       [array([1, 3]), 25137, 24821, 24915